# keras-nlp installation

In [1]:
!pip install -q git+https://github.com/keras-team/keras-nlp.git tensorflow --upgrade

  Preparing metadata (setup.py) ... done


# Imports

In [2]:
import keras_nlp
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

#Data

In [3]:
# Load Dataset

train_ds, valid_ds = tfds.load(
    "glue/sst2",
    split=["train", "validation"],
    batch_size=16,
)


def split_features(x):
    # GLUE comes with dictonary data, we convert it to a uniform format
    # (features, label), where features is a tuple consisting of all
    # features.
    features = x["sentence"]
    label = x["label"]
    return (features, label)


train_ds = train_ds.map(
    split_features, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.map(
    split_features, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

In [4]:
# Look first training set batch
# The format is (string_tensor, label_tensor)
train_ds.take(1).get_single_element()

(<tf.Tensor: shape=(16,), dtype=string, numpy=
 array([b'for the uninitiated plays better on video with the sound ',
        b'like a giant commercial for universal studios , where much of the action takes place ',
        b'company once again dazzle and delight us ',
        b"'s no surprise that as a director washington demands and receives excellent performances , from himself and from newcomer derek luke ",
        b', this cross-cultural soap opera is painfully formulaic and stilted . ',
        b", the film is n't nearly as downbeat as it sounds , but strikes a tone that 's alternately melancholic , hopeful and strangely funny . ",
        b'only masochistic moviegoers need apply . ',
        b'convince almost everyone that it was put on the screen , just for them ',
        b"like the english patient and the unbearable lightness of being , the hours is one of those reputedly `` unfilmable '' novels that has bucked the odds to emerge as an exquisite motion picture in its own righ

#Model

In [5]:
# Create the Classifier Model
# For more details please look https://keras.io/guides/keras_nlp/getting_started/

classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", num_classes=2, dropout=0.1
)

# Add loss function, optimizer and metrics
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.experimental.AdamW(5e-5),
    metrics=keras.metrics.SparseCategoricalAccuracy(),
    jit_compile=True,
)

# To see the summary(layers) of the model we need to build it(or call it once) but we don't need
# to do it if we just want to train it on a downstream task.
classifier.layers[-1].build(
    [
        None,
    ]
)
classifier.summary()

Model: "bert_classifier"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 padding_mask (InputLayer)      [(None, None)]       0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, None)]       0           []                               
                                                                                                  
 token_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 bert_backbone (BertBackbone)   {'sequence_output':  4385920     ['padding_mask[0][0]',           
                                 (None, None, 128),               'segment_ids[0][0]

In [6]:
# Train the model

N_EPOCHS = 2
classifier.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=N_EPOCHS,
)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Epoch 1/2
4210/4210 [==============================] - 828s 191ms/step - loss: 0.3782 - sparse_categorical_accuracy: 0.8299 - val_loss: 0.4344 - val_sparse_categorical_accuracy: 0.8165
Epoch 2/2
4210/4210 [==============================] - 783s 186ms/step - loss: 0.2409 - sparse_categorical_accuracy: 0.9039 - val_loss: 0.4626 - val_sparse_categorical_accuracy: 0.8222


In [7]:
# Save the weights

model_name = f"bert_tiny_uncased_en_sst2-epochs={N_EPOCHS}.h5"
classifier.save_weights(f"/content/{model_name}")

In [8]:
# Load the weigths

weights_path = f"bert_tiny_uncased_en_sst2-epochs={N_EPOCHS}.h5"
classifier.load_weights(weights_path)

#Test the model

In [9]:
# We will shuffle the valid dataset and take 1st example

shuffled_valid_ds = valid_ds.shuffle(55).rebatch(1)
element = shuffled_valid_ds.take(1).get_single_element()

pred_logits = classifier.predict(element[0])
pred = tf.argmax(pred_logits)

print(
    f"Text :: {element[0].numpy()[0].decode('utf-8')} \nLabel :: {element[1].numpy()[0]} "
    f"\nModel Prediction :: {tf.argmax(pred_logits, axis=1).numpy()[0]}"
)

1/1 [==============================] - 3s 3s/step
Text :: preaches to two completely different choirs at the same time , which is a pretty amazing accomplishment .  
Label :: 1 
Model Prediction :: 1


In [10]:
# You can test the model with your own statement!

label_dict = {0: "Bad Statement", 1: "Good Statement"}  # As for this dataset

output = tf.argmax(
    classifier.predict(["Lord of the rings is best"]), axis=1
).numpy()[0]
print(label_dict[output])

1/1 [==============================] - 1s 902ms/step
Good Statement
